In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Ερώτημα 1

In [2]:
def load_data(file_path):
    return pd.read_csv(file_path, sep='\t', header=None)

# Load the training and testing data
classA_train = load_data("/content/gdrive/My Drive/classA_train.dat")
classB_train = load_data("/content/gdrive/My Drive/classB_train.dat")
classA_test = load_data("/content/gdrive/My Drive/classA_test.dat")
classB_test = load_data("/content/gdrive/My Drive/classB_test.dat")

# Label the data
classA_train['Label'] = 0
classB_train['Label'] = 1
classA_test['Label'] = 0
classB_test['Label'] = 1

In [3]:
df = pd.DataFrame(classA_train)

featuresA= df[[0, 1, 2]].to_numpy()

featuresA.shape

df = pd.DataFrame(classB_train)

featuresB = df[[0, 1, 2]].to_numpy()

featuresB.shape

(20, 3)

In [4]:
df = pd.DataFrame(classA_train)

labelsA = df['Label'].to_numpy()

labelsA.shape


df = pd.DataFrame(classB_train)

labelsB = df['Label'].to_numpy()
featuresB.shape

(20, 3)

In [5]:
df = pd.DataFrame(classA_test)

featuresAT= df[[0, 1, 2]].to_numpy()

featuresA.shape

df = pd.DataFrame(classB_test)

featuresBT = df[[0, 1, 2]].to_numpy()

featuresB.shape


df = pd.DataFrame(classA_test)

labelsAT = df['Label'].to_numpy()

labelsA.shape


df = pd.DataFrame(classB_test)

labelsBT = df['Label'].to_numpy()

labelsB.shape

(20,)

In [6]:
train_data = np.concatenate((featuresA, featuresB), axis=0)
train_labels = np.concatenate((labelsA, labelsB), axis=0)
test_data = np.concatenate((featuresAT, featuresBT), axis=0)
test_labels = np.concatenate((labelsAT, labelsBT), axis=0)

In [7]:
pip install pennylane --upgrade


In [9]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

# Quantum device with five qubits
dev = qml.device("lightning.qubit", wires=3)

def layer(layer_weights):
    # Arbitrary rotation on each of the five qubits
    for wire in range(3):
        qml.Rot(*layer_weights[wire], wires=wire)

    # Ring of CNOTs for entanglement across all five qubits
    for wire_pair in ([0, 1], [1, 2], [2, 0]):
        qml.CNOT(wires=wire_pair)

def state_preparation(x):
    # Applying rotations based on the input x to each qubit
    for i, value in enumerate(x):
        qml.RY(np.pi * value, wires=i)



@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)
    for layer_weights in weights:
        layer(layer_weights)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Accuracy metric
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    return acc / len(labels)

def square_loss(labels, predictions):
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

# Cost function
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

X_train, Y_train = train_data, train_labels
Y_train = Y_train * 2 - 1


np.random.seed(0)
num_qubits = 3
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3)
bias_init = np.array(0.0)

opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

weights = weights_init
bias = bias_init
for it in range(100):
    # Update weights and bias using optimizer
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute predictions for the entire training set
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    current_cost = cost(weights, bias, X_train, Y_train)
    acc = accuracy(Y_train, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")


X_test = test_data
Y_test = test_labels
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

Iter:    1 | Cost: 2.4714397 | Accuracy: 0.4000000
Iter:    2 | Cost: 2.6087861 | Accuracy: 0.4000000
Iter:    3 | Cost: 2.5802856 | Accuracy: 0.4000000
Iter:    4 | Cost: 2.3958069 | Accuracy: 0.4000000
Iter:    5 | Cost: 2.1736472 | Accuracy: 0.5000000
Iter:    6 | Cost: 1.0957420 | Accuracy: 0.5000000
Iter:    7 | Cost: 1.1368053 | Accuracy: 0.5000000
Iter:    8 | Cost: 3.6106683 | Accuracy: 0.5000000
Iter:    9 | Cost: 1.5171200 | Accuracy: 0.4250000
Iter:   10 | Cost: 0.9411300 | Accuracy: 0.5750000
Iter:   11 | Cost: 1.6794954 | Accuracy: 0.6000000
Iter:   12 | Cost: 1.1698998 | Accuracy: 0.5000000
Iter:   13 | Cost: 1.0560830 | Accuracy: 0.5000000
Iter:   14 | Cost: 1.2127564 | Accuracy: 0.5000000
Iter:   15 | Cost: 1.0890776 | Accuracy: 0.5000000
Iter:   16 | Cost: 1.9395014 | Accuracy: 0.4000000
Iter:   17 | Cost: 1.8680441 | Accuracy: 0.4000000
Iter:   18 | Cost: 1.9620729 | Accuracy: 0.4000000
Iter:   19 | Cost: 1.9646864 | Accuracy: 0.4000000
Iter:   20 | Cost: 2.1143730 | 

#Ερώτημα 3

In [14]:
def load_data(file_path):
    return pd.read_csv(file_path, sep='\t', header=None)

# Load the training and testing data
classA_train = load_data("/content/gdrive/My Drive/classA_train_N5.dat")
classB_train = load_data("/content/gdrive/My Drive/classB_train_N5.dat")
classA_test = load_data("/content/gdrive/My Drive/classA_test_N5.dat")
classB_test = load_data("/content/gdrive/My Drive/classB_test_N5.dat")

# Label the data
classA_train['Label'] = 0
classB_train['Label'] = 1
classA_test['Label'] = 0
classB_test['Label'] = 1

df = pd.DataFrame(classA_train)

featuresA= df[[0, 1, 2,3,4]].to_numpy()

featuresA.shape

df = pd.DataFrame(classB_train)

featuresB = df[[0, 1, 2,3,4]].to_numpy()

featuresB.shape

df = pd.DataFrame(classA_train)

labelsA = df['Label'].to_numpy()

labelsA.shape


df = pd.DataFrame(classB_train)

labelsB = df['Label'].to_numpy()
featuresB.shape

df = pd.DataFrame(classA_test)

featuresAT= df[[0, 1, 2,3,4]].to_numpy()

featuresA.shape

df = pd.DataFrame(classB_test)

featuresBT = df[[0, 1, 2,3,4]].to_numpy()

featuresB.shape


df = pd.DataFrame(classA_test)

labelsAT = df['Label'].to_numpy()

labelsA.shape


df = pd.DataFrame(classB_test)

labelsBT = df['Label'].to_numpy()

labelsB.shape

train_data = np.concatenate((featuresA, featuresB), axis=0)
train_labels = np.concatenate((labelsA, labelsB), axis=0)
test_data = np.concatenate((featuresAT, featuresBT), axis=0)
test_labels = np.concatenate((labelsAT, labelsBT), axis=0)

In [15]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

# Quantum device with five qubits
dev = qml.device("lightning.qubit", wires=5)

def layer(layer_weights):
    # Arbitrary rotation on each of the five qubits
    for wire in range(5):
        qml.Rot(*layer_weights[wire], wires=wire)

    # Ring of CNOTs for entanglement across all five qubits
    for wire_pair in ([0, 1], [1, 2], [2, 3], [3,4], [4,0]):
        qml.CNOT(wires=wire_pair)

def state_preparation(x):
    # Applying rotations based on the input x to each qubit
    for i, value in enumerate(x):
        qml.RY(np.pi * value, wires=i)



@qml.qnode(dev)
def circuit(weights, x):
    state_preparation(x)
    for layer_weights in weights:
        layer(layer_weights)
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

# Accuracy metric
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    return acc / len(labels)

def square_loss(labels, predictions):
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

# Cost function
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

X_train, Y_train = train_data, train_labels
Y_train = Y_train * 2 - 1


np.random.seed(0)
num_qubits = 5
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3)
bias_init = np.array(0.0)

opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

weights = weights_init
bias = bias_init
for it in range(100):
    # Update weights and bias using optimizer
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute predictions for the entire training set
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    current_cost = cost(weights, bias, X_train, Y_train)
    acc = accuracy(Y_train, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")


X_test = test_data
Y_test = test_labels
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)


X_test = test_data
Y_test = test_labels
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

Iter:    1 | Cost: 2.1210660 | Accuracy: 0.4937500
Iter:    2 | Cost: 2.1826482 | Accuracy: 0.4937500
Iter:    3 | Cost: 1.8840404 | Accuracy: 0.4937500
Iter:    4 | Cost: 2.0003586 | Accuracy: 0.5187500
Iter:    5 | Cost: 0.8405804 | Accuracy: 0.7375000
Iter:    6 | Cost: 0.9764800 | Accuracy: 0.5000000
Iter:    7 | Cost: 1.0291325 | Accuracy: 0.5000000
Iter:    8 | Cost: 0.7725324 | Accuracy: 0.7000000
Iter:    9 | Cost: 0.9725371 | Accuracy: 0.7312500
Iter:   10 | Cost: 1.6241802 | Accuracy: 0.5000000
Iter:   11 | Cost: 0.8584280 | Accuracy: 0.5625000
Iter:   12 | Cost: 1.1960367 | Accuracy: 0.5000000
Iter:   13 | Cost: 1.0155524 | Accuracy: 0.5062500
Iter:   14 | Cost: 1.1199128 | Accuracy: 0.5000000
Iter:   15 | Cost: 0.7510164 | Accuracy: 0.7625000
Iter:   16 | Cost: 0.4979719 | Accuracy: 0.8062500
Iter:   17 | Cost: 0.3484327 | Accuracy: 0.9062500
Iter:   18 | Cost: 0.2514139 | Accuracy: 1.0000000
Iter:   19 | Cost: 0.1266786 | Accuracy: 1.0000000
Iter:   20 | Cost: 0.0243685 | 